In [20]:
import time
import datetime
import random
import numpy as np
from enum import Enum
from tabulate import tabulate as tb

#Global variables
user = "X"
AI = "O"
empty = "_"
timer = 3

class TicTacToeState(Enum):
    draw = "Draw"
    progress = "Playing"
    x = "X"
    o = "O"

#======Class for the Tic Tac Toe Game Board==========
class GameBoard:
    """
    Parameters:
    - Size: The game board size
    - Board: the board itself with the parameter size
    - finalmove: the final move on the board
    
    """
    def __init__(self, size):
        self.size = size
        self.board = [["_" for _ in range(size)] for _ in range(size)]
        self.finalMove = None

  # A method for displaying the board
    def displayBoard(self):
        print("Use the reference number on the board to make a move. ")
        #Reference board with numbers, so it is easier for the player to choose corresponding cell number to make a move
        reference_board = np.arange(self.size**2).reshape(self.size, -1)
        for x in range(self.size):
            for y in range(self.size):
                if y < self.size - 1:
                    print(reference_board[x][y], end="|")
                else:
                    print(reference_board[x][y], end="")
            print()

        print(tb(self.board, tablefmt="rounded_grid"))

    # Gets rows and columns
    def getRow(self, nr):
        return self.board[nr]

    def getCol(self, nc):
        return [r[nc] for r in self.board]

    #Gets the correct cell based on user input number.
    def getCell(self, cell):
        r, c = cell // self.size, cell % self.size
        return r, c

    # Gets the board diagonals
    def getEntireDiagonals(self):
        y = 0
        d1 = [self.board[x][x] for x in range(self.size)]
        d2 = []
        for x in list(range(self.size)[::-1]): 
            d2.append(self.board[x][y])
            y += 1
        return d1, d2

    def getMainDiagonals(self):
        return [self.board[x][x] for x in range(self.size)]

    #Checks if the cell is on the main board diagonals
    def isOnMainDiagonals(self, cell):
        return cell % (self.size + 1) == 0

    def getAltDiag(self):
        y, d2 = 0, []
        for x in list(range(self.size)[::-1]):
            d2.append(self.board[x][y])
            y += 1
        return d2

    def isOnAltDiag(self, cell):
        return cell % (self.size - 1) == 0
    
    #Gets cell positions on the board
    def NoMove(self, cell):
        r, c = self.getCell(cell)
        self.board[r][c] = "_"

    def PlayerMove(self, cell):
        self.finalMove = cell
        r, c = self.getCell(cell)
        self.board[r][c] = "X"

    def AIMove(self, cell):
        self.finalMove = cell
        r, c = self.getCell(cell)
        self.board[r][c] = "O"
    
    def getFinalMove(self):
        return self.finalMove

    #Checks if a line on the board is filled with same chars
    def isLineFilledSame(self, check, m):
        return all(i == m for i in check)

    #Checks if the cell is empty
    def isCellEmpty(self, cell):
        r, c = self.getCell(cell)
        return self.board[r][c] == "_"
    

#=======Exceptions=========
#The move should be integer
class NotInteger(Exception):
    pass

# when user enters a move in an already filled cell
class FilledCell(Exception):
    pass

#The move should be within the board size
class GoesOverBoardSize(Exception):
    pass

#==============
class TicTacToe:
    """
    Class for the Tic Tac Toe game.
    
    Args:
        BoardSize (Integer):The board size N. 
        AItimer (Integer): Time for the AI to make a move (in seconds).
        users (Integer):The number of users
        simulate (Boolean): True if we simulate the game, False if a user plays by making move choices on their own.
        display (Boolan): Displaying the board
    """

    def __init__(self, BoardSize, AItimer, users, simulate, display):
        self.BoardSize = BoardSize
        self.AItimer = AItimer
        self.users = users
        self.simulate = simulate
        self.display = display
        self.ticBoard = GameBoard(BoardSize)
        self.usernames = [' '] * users
        self.turn = None  
        self.FirstMoveRandom()
        self.bestMove = 0
        self.AIFirstMove = None
        
        
    #If the game is not a simulation, the user will be prompted to enter their player name         
    def getUsername(self):
        usersize = 1
        while usersize <= self.users:
            try:
                if self.simulate == False:
                    username = input(
                        "Please Enter Player " + str(usersize) + "'s Name: ")
                elif self.simulate == True:
                    username = "Player"
                if username is None:
                    raise ValueError("Please enter your name.")
                self.usernames[usersize - 1] = username
                usersize += 1
            except ValueError as i:
                print(i)

    #Decides the first move maker randomly
    def FirstMoveRandom(self):
        FirstMoveMaker = np.random.choice(["Player", "AI"])
        if FirstMoveMaker == "Player":
            self.turn = 0 #Player's turn is 0
            if self.display == True:
                print("You are randomly chosen to make the first move.")
        else:
            self.AIFirstMove = random.randrange(
                self.ticBoard.size ** 2)
            self.turn = 1
            if self.display == True:
                print("AI is randomly chosen to make the first move")
    
    #Gets the list of possible valid moves to make
    def getValidMoves(self):
        valid_moves = []
        for cell in range(self.ticBoard.size ** 2):
            if self.ticBoard.isCellEmpty(cell):
                valid_moves.append(cell)
        return valid_moves
    
    #Gets the next move
    def getNextMove(self):
        while True:
            try:
                #If the player is playing and it is their turn, prompt them to make a move
                if self.simulate == False:
                    userMove = int(
                        input(self.usernames[self.turn] + " Choose a cell to make your move: "))
                #If it is simulation, randomly get moves from valid moves
                elif self.simulate == True:
                    if self.display:
                        print("Player is making a move.")
                    userMove = int(np.random.choice(self.getValidMoves()))
                #Exceptions
                if not isinstance(userMove, int):
                    raise NotInteger("Please enter a number")
                if not self.ticBoard.isCellEmpty(userMove):
                    raise FilledCell(
                        "This cell is already filled. Choose a different cell.")
                if not (0 <= userMove <= (self.BoardSize ** 2 - 1)):
                    raise GoesOverBoardSize(
                        "Please make a move within the board size.")
                return userMove
            except(NotInteger, FilledCell, GoesOverBoardSize) as i:
                print(i)
            except Exception:
                print("Error. Please choose a valid move.")

    #=====Checking=======
    #Whether it is a draw or not. True if draw.
    def checkDraw(self):
        for cell in range(self.ticBoard.size ** 2):
            if self.ticBoard.isCellEmpty(cell):
                return False
        return True

    #Whether it is a win or not. 
    def checkWin(self, move):
        move_maker = ""
        if move % 2 == 0:
            move_maker = user #the global variable
        else:
            move_maker = AI #the global variable
        final_move = self.ticBoard.getFinalMove()
        r, c = self.ticBoard.getCell(final_move)
        #Checks if a row or a column is filled with same chars, if yes - win. 
        if self.ticBoard.isLineFilledSame(self.ticBoard.getRow(r), move_maker) or self.ticBoard.isLineFilledSame(
                self.ticBoard.getCol(c), move_maker):
            return True
        #Checks if the diagonals is filled with same chars and the final move is on the main diagonals, if yes - win
        if self.ticBoard.isOnMainDiagonals(final_move) and self.ticBoard.isLineFilledSame(self.ticBoard.getMainDiagonals(), move_maker):
            return True
        if self.ticBoard.isOnAltDiag(final_move) and self.ticBoard.isLineFilledSame(self.ticBoard.getAltDiag(), move_maker):
            return True
        return False

    def checkTicTacToeState(self):
        if self.checkDraw():
            return "Draw"
        if self.checkWin(0):
            return "X"
        if self.checkWin(1):
            return "O"
        return "Playing"


    #====Starting the game====
    def StartGame(self):
        result = 0 
        self.getUsername()
        while True:
            if self.display:
                self.ticBoard.displayBoard()
            self.turn %= 2
            if self.turn % 2 == 0:
                userMove = self.getNextMove()
                self.ticBoard.PlayerMove(userMove)
            else:
                #AI making the move
                if self.AIFirstMove is not None:
                    ai_move = self.AIFirstMove
                    self.AIFirstMove = None
                else:
                    #Does iterative Deepening search for the Ai moves
                    ai_move = self.iterativeDeepening()
                self.ticBoard.AIMove(ai_move)
            
            #registers the current state of the game
            current = self.checkTicTacToeState()
            
            #If the game has ended:
            if current != "Playing":
                if self.display:
                    self.ticBoard.displayBoard()
                    
                #Return 0 if draw, -1 if user wins, 1 if AI wins
                if current == "Draw":
                    if self.display == True:
                        print("GAME RESULT: DRAW!")
                    result = 0
                else:
                    if self.turn % 2 == 0:
                        if self.display == True:
                            print("GAME RESULT: ",
                                  self.usernames[self.turn], "YOU WON!")
                        result = -1
                    else:
                        if self.display == True:
                            print("GAME RESULT: AI WON!")
                        result = 1
                return result
                break
            #If the game is still playing, move on to the next move.
            self.turn += 1

    #=====This part is to get point for each of the line in the board=========
    # Checks the number of O, X or _ is there in a line
    def checkLine(self, line):
        return line.count("O"), line.count("X"), line.count("_")

    #By checking the line, appoints a point for a single line
    def getLinePoints(self, line):
        aiPoints, userPoints, emptyPoints = self.checkLine(line)
        points = 0
        if aiPoints == 0 and userPoints != 0:
            points += -(10 ** (userPoints - 1))
        if userPoints == 0 and aiPoints != 0:
            if aiPoints == self.ticBoard.size:
                points += 11 ** (aiPoints - 1)
            points += 10 ** (aiPoints - 1)
        return points

    # Method which returns board points after evaluation of every possible line.
    def getBoardPoint(self):
        points = 0
        d = self.ticBoard.getEntireDiagonals()
        for cell in range(self.ticBoard.size):
            points += self.getLinePoints(self.ticBoard.getRow(cell))
            points += self.getLinePoints(self.ticBoard.getCol(cell))
        for cell in range(2):
            points += self.getLinePoints(d[cell])
        return points

    #====== Minimax with alpha-beta pruning =========
    def minimax(self, height, Max, alpha, beta, start, maxtime):
        """
        Parameters:
        ---------------
        - Height: The height of the search tree
        - Max: True if it is maximizing player, False if minimizing
        - Alpha: The best value for Max
        - Beta: The best value for Min
        - Start: The time that AI starts the search
        - Maxtime: The maximum time that the AI has to get the best move
        """
        validmoves = self.getValidMoves() #The list of valid moves
        points = self.getBoardPoint()
        cell = None
        
        if datetime.datetime.now() - start >= maxtime:
            self.timeOver = True
        
        if not validmoves or height == 0 or self.timeOver:
            current = self.checkTicTacToeState()
            if current== "X":
                return -10 ** (self.ticBoard.size + 1), cell
            elif current == "O":
                return 10 ** (self.ticBoard.size + 1), cell
            elif current == "Draw":
                return 0, cell
            return points, cell

        if Max == True:
            for x in validmoves:
                self.ticBoard.AIMove(x)
                points, temp = self.minimax(
                    height - 1, not Max, alpha, beta, start, maxtime)
                if points > alpha:
                    alpha = points
                    cell, self.bestMove = x, x
                self.ticBoard.NoMove(x)
                if beta <= alpha:
                    break
            return alpha, cell

        else:
            for x in validmoves:
                self.ticBoard.PlayerMove(x)
                points, temp = self.minimax(
                    height - 1, not Max, alpha, beta, start, maxtime)
                if points < beta:
                    beta = points
                    cell, self.bestMove = x, x
                self.ticBoard.NoMove(x)
                if alpha >= beta:
                    break
            return beta, cell
    

    # =====Iterative Deepening Search==========
    def iterativeDeepening(self):
        height, cell, self.timeOver = 1, None, False
        StartTime = datetime.datetime.now()
        endTime = StartTime + datetime.timedelta(0, self.AItimer)
        while True:
            runtime = datetime.datetime.now()
            if runtime >= endTime:
                break
            maxi, cell = self.minimax(
                height, False, -10000000, 10000000, runtime, endTime - runtime)
            height += 1
        if cell == None:
            cell = self.bestMove
        return cell


In [37]:
#TicTacToe(self, BoardSize, AItimer, users, simulate, display)
game = TicTacToe(4, 5, 1, True, True)
start = time.time() #starting timer
game.StartGame()
print(f"It took {time.time()-start:.3f} seconds to finish") #How many seconds did it take to finish?



AI is randomly chosen to make the first move
Use the reference number on the board to make a move. 
0|1|2|3
4|5|6|7
8|9|10|11
12|13|14|15
╭───┬───┬───┬───╮
│ _ │ _ │ _ │ _ │
├───┼───┼───┼───┤
│ _ │ _ │ _ │ _ │
├───┼───┼───┼───┤
│ _ │ _ │ _ │ _ │
├───┼───┼───┼───┤
│ _ │ _ │ _ │ _ │
╰───┴───┴───┴───╯
Use the reference number on the board to make a move. 
0|1|2|3
4|5|6|7
8|9|10|11
12|13|14|15
╭───┬───┬───┬───╮
│ _ │ _ │ _ │ O │
├───┼───┼───┼───┤
│ _ │ _ │ _ │ _ │
├───┼───┼───┼───┤
│ _ │ _ │ _ │ _ │
├───┼───┼───┼───┤
│ _ │ _ │ _ │ _ │
╰───┴───┴───┴───╯
Player is making a move.
Use the reference number on the board to make a move. 
0|1|2|3
4|5|6|7
8|9|10|11
12|13|14|15
╭───┬───┬───┬───╮
│ _ │ _ │ _ │ O │
├───┼───┼───┼───┤
│ _ │ _ │ _ │ _ │
├───┼───┼───┼───┤
│ _ │ _ │ X │ _ │
├───┼───┼───┼───┤
│ _ │ _ │ _ │ _ │
╰───┴───┴───┴───╯
Use the reference number on the board to make a move. 
0|1|2|3
4|5|6|7
8|9|10|11
12|13|14|15
╭───┬───┬───┬───╮
│ _ │ _ │ _ │ O │
├───┼───┼───┼───┤
│ _ │ O │ _ │ _ │
